# Splitting dataset into train, test and valid subsets 

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import shutil

In [14]:
import pandas as pd
import numpy as np
import os
import shutil

def split_dataset_and_files(metadata_path, audio_folder, output_folder, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Podziel zbiór danych na zbiory treningowy, walidacyjny i testowy
    z uwzględnieniem wymagań dotyczących artystów i utworów oraz przenieś pliki audio
    do odpowiadających im folderów.
    """
    # Wczytaj dane
    metadata = pd.read_csv(metadata_path)

    # Sprawdzenie poprawności proporcji
    if not np.isclose(train_ratio + val_ratio + test_ratio, 1.0):
        raise ValueError("Proporcje zbiorów muszą sumować się do 1.0")
    
    # Grupuj dane po artystach
    artist_groups = metadata.groupby('artist_id')

    # Losowy podział artystów na zbiory
    artist_ids = list(artist_groups.groups.keys())
    np.random.shuffle(artist_ids)
    
    train_end = int(len(artist_ids) * train_ratio)
    val_end = train_end + int(len(artist_ids) * val_ratio)

    train_artists = artist_ids[:train_end]
    val_artists = artist_ids[train_end:val_end]
    test_artists = artist_ids[val_end:]

    # Tworzenie zbiorów
    train_data = metadata[metadata['artist_id'].isin(train_artists)]
    val_data = metadata[metadata['artist_id'].isin(val_artists)]
    test_data = metadata[metadata['artist_id'].isin(test_artists)]

    # Walidacja podziału
    assert len(train_data) + len(val_data) + len(test_data) == len(metadata), \
        "Suma zbiorów nie zgadza się z całkowitą liczbą danych"

    # Tworzenie folderów wyjściowych
    os.makedirs(os.path.join(output_folder, "train"), exist_ok=True)
    os.makedirs(os.path.join(output_folder, "val"), exist_ok=True)
    os.makedirs(os.path.join(output_folder, "test"), exist_ok=True)

    # Funkcja do kopiowania plików do odpowiednich folderów
    def move_files(data, folder):
        for _, row in data.iterrows():
            src = os.path.join(audio_folder, os.path.basename(row['path']))
            dest = os.path.join(output_folder, folder, os.path.basename(row['path']))
            if os.path.exists(src):
                shutil.copy(src, dest)
            else:
                print(f"Plik {src} nie istnieje i zostanie pominięty.")

    # Przenoszenie plików
    print("Przenoszenie plików do folderu 'train'...")
    move_files(train_data, "train")
    print("Przenoszenie plików do folderu 'val'...")
    move_files(val_data, "val")
    print("Przenoszenie plików do folderu 'test'...")
    move_files(test_data, "test")

    return train_data, val_data, test_data

In [15]:
# Ustawienia ścieżki i proporcji
metadata_path = "../../datasets/jamendo/metadata/processed_audio_metadata.csv"
audio_folder = "../../datasets/jamendo/processed_audio/"
output_folder = "../../datasets/jamendo/split_audio/"
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Podział danych i plików
train_data, val_data, test_data = split_dataset_and_files(metadata_path, audio_folder, output_folder, train_ratio, val_ratio, test_ratio)

# Zapis do plików CSV
train_data.to_csv("../../datasets/jamendo/metadata/train_metadata.csv", index=False)
val_data.to_csv("../../datasets/jamendo/metadata/val_metadata.csv", index=False)
test_data.to_csv("../../datasets/jamendo/metadata/test_metadata.csv", index=False)

print("Zbiory danych i pliki audio zostały podzielone i zapisane:")
print(f"- Zbiór treningowy: {len(train_data)} przykładów")
print(f"- Zbiór walidacyjny: {len(val_data)} przykładów")
print(f"- Zbiór testowy: {len(test_data)} przykładów")

Przenoszenie plików do folderu 'train'...
Przenoszenie plików do folderu 'val'...
Przenoszenie plików do folderu 'test'...
Zbiory danych i pliki audio zostały podzielone i zapisane:
- Zbiór treningowy: 32226 przykładów
- Zbiór walidacyjny: 11482 przykładów
- Zbiór testowy: 6299 przykładów
